In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import random

## PATH CONFIGURATION 

In [ ]:
# Set paths for original and output directories
EMIDEC_ORIGINAL_PATH = r"Set this to the directory where your localized slices are stored"
EMIDEC_CLIPPED_PATH = r"Set this to the directory where you want to store the clipped slices"


## Apply Clipping Based on Percentiles

In [ ]:
def clip_images(input_dir, output_dir, low_percentile=2, high_percentile=98):
    os.makedirs(output_dir, exist_ok=True)

    npy_files = [f for f in os.listdir(input_dir) if f.endswith(".npy")]
    all_pixels = []

    # Collect all pixel values for threshold calculation
    for file_name in npy_files:
        img = np.load(os.path.join(input_dir, file_name)).flatten()
        all_pixels.extend(img)

    all_pixels = np.array(all_pixels)
    low_thresh = np.percentile(all_pixels, low_percentile)
    high_thresh = np.percentile(all_pixels, high_percentile)
    print(f"{low_percentile}th Percentile: {low_thresh}")
    print(f"{high_percentile}th Percentile: {high_thresh}")

    # Apply clipping and save
    for file_name in npy_files:
        img = np.load(os.path.join(input_dir, file_name))
        clipped = np.clip(img, low_thresh, high_thresh)
        np.save(os.path.join(output_dir, file_name), clipped)

    # Plot histogram after clipping
    plt.figure(figsize=(10, 5))
    plt.hist(all_pixels[(all_pixels >= low_thresh) & (all_pixels <= high_thresh)],
             bins=100, color='blue', alpha=0.7)
    plt.title(f"Histogram After Clipping ({low_percentile}th & {high_percentile}th)")
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Frequency")
    plt.yscale("log")
    plt.show()


## Visual Comparison of Original vs Clipped

In [ ]:
def visualize_random_samples(original_dir, clipped_dir, num_samples=5):
    npy_files = [f for f in os.listdir(original_dir) if f.endswith(".npy")]
    random_files = random.sample(npy_files, num_samples)

    fig, axes = plt.subplots(num_samples, 2, figsize=(10, 3 * num_samples))

    for i, file_name in enumerate(random_files):
        orig_img = np.load(os.path.join(original_dir, file_name))
        clipped_img = np.load(os.path.join(clipped_dir, file_name))

        if orig_img.ndim == 3:
            slice_idx = orig_img.shape[0] // 2
            orig_slice = orig_img[slice_idx]
            clipped_slice = clipped_img[slice_idx]
        else:
            orig_slice = orig_img
            clipped_slice = clipped_img

        axes[i, 0].imshow(orig_slice, cmap="gray")
        axes[i, 0].set_title(f"Original: {file_name}")
        axes[i, 0].axis("off")

        axes[i, 1].imshow(clipped_slice, cmap="gray")
        axes[i, 1].set_title(f"Clipped: {file_name}")
        axes[i, 1].axis("off")

    plt.tight_layout()
    plt.show()


## Usage Example

In [ ]:
# Clip test data
clip_images(EMIDEC_ORIGINAL_PATH, EMIDEC_CLIPPED_PATH)

# Visualize test results
visualize_random_samples(EMIDEC_ORIGINAL_PATH, EMIDEC_CLIPPED_PATH)
